In [ ]:
import polars as pl
import polars.selectors as ps
import plotly.graph_objects as go
import plotly.express as px

from dataframe import load

dataframe = load("data.parquet").with_columns(
    is_fallen=pl.col("Control.main_outputs.robot_orientation.pitch").abs() > 1.0
)

In [ ]:
robot_identifier = "10.1.24.32"

px.scatter(
    dataframe.filter(pl.col("robot_identifier") == robot_identifier),
    x="time",
    y="Control.main_outputs.robot_orientation.pitch",
    # color="is_fallen",
    color = "Control.main_outputs.fall_state",
)

In [ ]:
from sklearn.neighbors import LocalOutlierFactor

selector = ps.contains("robot_orientation")
df = dataframe.drop_nulls(selector)
lof = LocalOutlierFactor(n_neighbors=200)
lof_value = lof.fit_predict(df.select(selector))
df = df.with_columns(lof=pl.Series("lof", lof_value))

In [ ]:
robot_identifier = "10.1.24.32"

px.scatter(
    df.filter(pl.col("robot_identifier") == robot_identifier),
    x="time",
    y="Control.main_outputs.robot_orientation.pitch",
    # color="is_fallen",
    color = "lof",
)

In [ ]:
df = dataframe
x = pl.col("Control.main_outputs.zero_moment_point.x")
y = pl.col("Control.main_outputs.zero_moment_point.y")
df = df.with_columns(
    (x ** 2 + y ** 2).sqrt().alias("distance"),
)
px.scatter(
    df.filter(pl.col("robot_identifier") == robot_identifier),
    x="time",
    y="distance",
    color="is_fallen",
    # color = "Control.main_outputs.fall_state",
)